# Benchmark AMR-D1 creation

Amazon Movie Reviews - Drift 1


In [4]:
# Reload modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

# Import from code root directory
import sys; sys.path.insert(0, '../')

# Imports opinion words
from access.file_storage import FileStorage
from rarfile import RarFile
import os

# Imports Amazon
#from access.amazon_pickle_reader import AmazonPickleReader
from access.amazon_pickle_splitter import AmazonPickleSplitter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
file_storage = FileStorage()

## Opinion words

https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#lexicon

In [6]:
# Config
opinion_words_id = 'opinion-words'
print(file_storage.get_filepath(opinion_words_id))

opinion_words_directoy = os.path.dirname(file_storage.get_filepath(opinion_words_id))
print(opinion_words_directoy)

negative_words_file = os.path.join(opinion_words_directoy, 'opinion-lexicon-English', 'negative-words.txt')
positive_words_file = os.path.join(opinion_words_directoy, 'opinion-lexicon-English', 'positive-words.txt')
print(negative_words_file)
print(positive_words_file)

/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/opinion-words/opinion-lexicon-English.rar
/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/opinion-words
/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/opinion-words/opinion-lexicon-English/negative-words.txt
/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/opinion-words/opinion-lexicon-English/positive-words.txt


In [7]:
# Download opinion words, if not already exists
file_storage.download(opinion_words_id)

File exists, will not be downloaded


In [8]:
# Extract file, if not already extracted
if not os.path.exists(negative_words_file):
    with RarFile(file_storage.get_filepath(opinion_words_id)) as file:
        file.extractall(opinion_words_directoy)

In [9]:
# Read words files
def read_opinion_file(file):
    words = []
    with open(file, 'r', encoding="iso-8859-1") as f:
        for line in f:
            if line and line.strip() and not line.startswith(';'):
                words.append(line.strip())
    return words
negative_words = read_opinion_file(negative_words_file)
positive_words = read_opinion_file(positive_words_file)
print(len(negative_words))
print(len(positive_words))

4783
2006


## Amazon

In [10]:
# Config
amazon_raw_id = 'amazon_raw.pickle'
print(file_storage.get_filepath(amazon_raw_id))

amazon_directoy = os.path.dirname(file_storage.get_filepath(amazon_raw_id))
print(amazon_directoy)

/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/clustering/amazon_raw.pickle
/home/eml4u/EML4U/notebooks/ExplainingDriftTextEmbeddings/data/clustering


In [11]:
# Read data
amazon_splitter = AmazonPickleSplitter(amazon_directoy)

In [15]:
# Print overview
if False:
    amazon_splitter.print_overview()
    
# 1997	1998	1999	2000	2001	2002	2003	2004	2005	2006	2007	2008	2009	2010	2011	2012	
# 6		191		4844	19944	24221	25311	25734	41016	54744	49049	49521	56076	59099	65343	72957	81276	0
# 1		262		3631	17808	20320	22641	24183	33117	40868	37992	40205	40138	39680	41430	45767	47356	1
# 8		442		6458	30907	35395	37798	43323	60489	71012	66128	75239	74057	73178	70279	72055	74826	2
# 29	797		14178	73314	79152	84276	90527	119160	138000	135581	167632	161693	149771	142000	148457	150248	3
# 64	3313	49866	192002	189638	198712	205916	257603	308080	311252	452009	412870	422403	426248	465918	484650	4

# Idea

- Two distributions 1-star and 5-star data: One original, one mixed -> Artificial drift
- Use 2001 to inject 10%, 2002 to inject 20% and so on
- Sort by positive/negative words to exclude low quality reviews by users
- What's required? [training, validation, test](https://en.wikipedia.org/wiki/Training%2C_validation%2C_and_test_sets). Sizes of each?

In [ ]:
# c =  len([i for i in list2 if i in list1 ])